# First Aid Kit

It's a pity, but it is impossible to develop applications that work perfectly.
Applications always contain errors due to programming mistakes, incorrect use of function calls, or external reasons, such as a missing file or the abnormal termination of an external (non-APL) piece of code.

All these problems may interrupt the normal processing of an application, and require immediate corrective action.
The aim of this chapter is to give you the basis to deal with these situations, and work through the experience.

In [a later chapter](./Event-Handling.ipynb) you will see that it is possible to trap and hide unpredictable errors or events, in order to provide a safe environment for the user.

Because APL is an interpreted language, program execution is not aborted, just suspended when an exception occurs.
Then the programmer has access to all the application variables, and can diagnose the problem.
If they can modify the code or correct some environmental dependency, they may be able to restart the program from the very point it had been interrupted in, until a new exception occurs, and so on.
This gives great flexibility when debugging applications.

In this chapter you will learn how to:
 - interpret error messages;
 - interpret the information provided by various system indicators;
 - trace the execution of an application step by step; and
 - set *breakpoints* in strategic places to help debugging.
 
A forgotten comma, an incorrect axis, a missing parenthesis, a scalar where you expected a vector, can all cause an error and it may sometimes be very difficult for a beginner to understand the reason.
Try to get help from people who have a better understanding of APL:
you will save time *and* learn a lot from them.

## When an Error Occurs

### Our First Error

Even when you use a function, that is in itself perfectly correct, errors may occur if you call it with arguments that are not consistent with the function's requirements.
We will use this type of mistake to illustrate what happens when an error occurs.

In the previous chapter we wrote a function to interlace two matrices of the same shape:

In [1]:
∇ r ← x Interlace y; size; even
    size ← 1 2×⍴x
    r ← size⍴0
    even ← 2×⍳(⍴x)[2]
    r[;even] ← y
    r[;even-1] ← x
∇

In [2]:
(2 2⍴1) (2 2⍴0)

┌───┬───┐
│1 1│0 0│
│1 1│0 0│
└───┴───┘

In [3]:
(2 2⍴1) Interlace (2 2⍴0)

1 0 1 0
1 0 1 0

This function works perfectly if you stick to its rules, but what happens if you call it with matrices that *don't* have the same shape?

#### Your Environment and Indicators

Let us try to interlace our `forecast` matrix, that has 4 rows and 6 columns,

In [4]:
⎕RL ← 73
⎕←forecast ← 10×?4 6⍴55

90 160 420 500 20 30
110 450 170 370 290 360
340 190 320 120 510 370
150 460 240 520 490 280

with a 3 by 5 matrix, and see what happens:

In [5]:
forecast Interlace 3 5⍴⍳15

LENGTH ERROR
Interlace[4] r[;even]←y
              ∧


The first thing we see is the error message that is issued.
Then, if you are using RIDE or the Windows interpreter, you will see that, in the lower right corner of the window, some information is highlighted.
<!--figure-->The first figure below<!--Win_Red_State_Indicator--> shows this highlighting in red, for users of the Windows interpreter, and <!--figure-->the next figure below<!--RIDE_Purple_State_Indicator--> shows this highlighting in purple for users of the RIDE interpreter.

![Red highlighting of `⎕SI` upon reaching an error in the Windows interpreter.](res/Win_Red_State_Indicator.png)

<br>

![Purple highlighting of `⎕SI` upon reaching an error in the RIDE interpreter.](res/RIDE_Purple_State_Indicator.png)

Furthermore, when using the Windows or RIDE interpreters, an additional trace windows appears, generally at the bottom, as shown respectively by <!--figure-->the next figures<!--Win_Trace_Window,RIDE_Trace_Window-->.

![Trace window in the Windows interpreter.](res/Win_Trace_Window.png)

---

![Trace window in the RIDE interpreter.](res/RIDE_Trace_Window.png)

APL will preserve the entire execution context (local variables and program status indicators), so that you can resume function execution once the error has been diagnosed and repaired.

Let us now examine the meaning of these things.

#### The Diagnostic Message

The *diagnostic message* is displayed on three rows of the screen which show:
 - the type of error (explained later in [this section](#Execution-Errors));
 - the function name [and line number] - the statement that can't be executed; and
 - a caret (the character `∧`) which is placed where the statement was interrupted.
 
A *system function* named `⎕DM` returns the latest *diagnostic message*.
It is a three-item nested vector, with a sub-vector for each of the three lines, as shown below:

In [6]:
)copy display DISPLAY
DISPLAY ⎕DM

C:\Program Files\Dyalog\Dyalog APL-64 18.0 Unicode\ws\display.dws saved Tue Dec 1 20:08:12 2020

┌→───────────────────────────────────────────────────────────┐
│ ┌→───────────┐ ┌→──────────────────────┐ ┌→──────────────┐ │
│ │LENGTH ERROR│ │Interlace[4] r[;even]←y│ │ ∧│ │
│ └────────────┘ └───────────────────────┘ └───────────────┘ │
└∊───────────────────────────────────────────────────────────┘

`⎕DM` retains this value until it is replaced by the occurrence of a new error, or until we explicitly clear it.

#### State Indicator and Line Counter

Whenever function execution is interrupted for any reason, Dyalog APL keeps track of the exact point where the interrupt occurred in the *state indicator* (SI for short).
This value of the state indicator may be obtained using a system command `)si` or a system function `⎕SI`:

 - system command:

```APL
      )si
#.Interlace[4]*
```

 - system function:

```APL
      ⎕SI
┌─────────┐
│Interlace│
└─────────┘
```

The *system function* `⎕SI` returns a nested vector containing only the names of the interrupted functions;
in this example only one name appears.

The *system command* `)si` reports, for each function in the execution stack:
 - a path (here `#.`) which will be explained in [a later chapter](./Namespaces.ipynb). Ignore it for now;
 - the function name (`Interlace`);
 - the number of the statement on which function execution has stopped (`[4]` in this case); and
 - an asterisk which means that this function has been interrupted.
 
What you see highlighted in the bottom right-hand corner of your session (`⎕SI:1`) is the number of functions referenced in the *state indicator*.
Normally, this number should be zero and displayed in black; as soon as an error or an interrupt occurs, it is displayed in a different colour as a warning (in red or purple, depending on your software).

APL also keeps, in a *line counter*, the list of function lines waiting for execution.
In this case there is only one.
It can be obtained from the *system function* `⎕LC`:

```APL
      ⎕LC
4
```

The number above shows that `Interlace` was interrupted on line 4.

<!-- begin note -->
***Note***:

 > Due to limitations of the Jupyter interface, many of the things we will be discussing here will work differently when applied directly to a Dyalog APL notebook.
<!-- end -->

For example, even though we got an error with the incorrect usage of `Interlace` and we were able to check the diagnostic message with `⎕DM`, the *state indicator* and the *line counter* are empty:

In [7]:
forecast Interlace 3 5⍴⍳15

LENGTH ERROR
Interlace[4] r[;even]←y
              ∧


In [8]:
⎕SI

In [9]:
⎕LC

#### The Trace Window

If you are running Dyalog APL using the default *tracer* behaviour, when you reach an error inside a function a window should pop up in the bottom of the interpreter, with the function text displayed in another colour and with the statement in error highlighted.

If you are using the Windows interpreter and the trace window did not appear spontaneously, just press <kbd>Ctr</kbd>+<kbd>Enter</kbd>. Then, go to "Options" &#8680; "Configuration" &#8680; "Trace/Edit" and tick the checkbox next to "*Show trace stack on error*".

We shall see later how this trace window can be used.
**Do not close it** for the moment.

#### You Can See the Local Variables

Your cursor may not be in this (trace) window: it may remain in the session window, so that you can conveniently enter expressions to diagnose what has happened.
If the trace window *does* have the focus, you can just switch the focus to the session window and work there, without closing the trace window.

Because the function has been interrupted, all its local variables are visible:
 - you can look at their values; and
 - you can even modify them (sometimes this may help).
 
```APL
      ⍴y
3 5
      size
4 12
      ⍴r[;even]
4 6
```

You can see that the size of the future result is correct, but we cannot assign the 3 rows of `y` into the 4 rows of `r`.

#### What Can We Do?

Obviously, in this case there is nothing we can do: the function is correct, we just misused it!
We could, perhaps, forget about it and go on to do something else.
However, if we do so, APL will retain the entire execution stack of functions and local variables in its interrupted state.
If we are not going to continue debugging, we should really clean up by getting the system out of this state.

There are three ways to achieve this.
If you want to try all three of them out, you will need to reproduce the same error after each exit operation:
 1. close the trace window. You can use the standard windows methods to close it, or you can click on it (to give it the focus) and then press <kbd>Esc</kbd> to close it;
 2. type a branch arrow `→` in the session window (as seen in [the Specialist Section on the niladic arrow](./User-Defined-Functions.ipynb#Niladic-Arrow)); or
 3. execute the `)reset` command in the session window.

Whichever option you chose, you can verify that:
 - both `)si` and `⎕SI` now give an empty result and
 - the highlighted indication `⎕SI:1` is replaced by a neutral `⎕SI:0` in the session status bar.

However, there is a little difference: with the first two methods, `⎕DM` retains its value (and keeps track of the latest error), while `)reset` also clears `⎕DM`.
In fact, we can generate an error right here in the session:

In [10]:
1 2 + 3 4 5

LENGTH ERROR: Mismatched left and right argument shapes
      1 2+3 4 5
         ∧


Notice `⎕DM` contains the information about this error:

In [11]:
⎕DM

┌────────────┬───────────────┬──────────┐
│LENGTH ERROR│ 1 2+3 4 5│ ∧│
└────────────┴───────────────┴──────────┘

If we use the `)reset` command, then `⎕DM` becomes empty:

In [12]:
)reset

In [13]:
⎕DM

### Cascade of Errors

We shall now see what happens when an error occurs in a sub-function.

#### Preparation

A palindrome is a string of characters which remains the same when it is reversed, after any non-alphabetic characters (including spaces) have been removed.
For example, "*Cigar: toss it in a can. It is so tragic.*" and "*Was it a car or a cat I saw?*" are palindromes.

We shall try to write a function which detects whether or not a given string of characters is a palindrome.
We are going to use two auxiliary functions:

 1. `Upper` removes all non-alphabetic characters and transforms the remaining letters into uppercase characters.

In [14]:
]dinput
Upper ← {
    low ← 'abcdefghijklmnopqrstuvwxyz'
    up  ← 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    all ← low,up
    text ← (⍵∊all)/⍵
    (up,up)[all⍳text]
}

In [15]:
Upper 'Visit New York'

VISITNEWYORK

 2. `Reverse` reverses the items of a vector.

This is one possible solution to an exercise from the previous chapter, written by someone who was unaware that APL has a *reverse* primitive (introduced in [a later chapter](./Working-on-Data-Shape.ipynb)):

In [16]:
]dinput
Reverse ← {
    index ← (1+≢⍵)-⍳≢⍵
    ⍵[index]
}

In [17]:
Reverse 'Demonstration'

noitartsnomeD

Finally, `Palindrome` compares the uppercase version of the original vector against its reversed version:

In [18]:
]dinput
Palindrome ← {
    vector ← Upper ⍵
    torvec ← Reverse vector
    torvec≡vector
}

Let's try it out:

In [19]:
Palindrome 'Was it a car or a cat I saw?'

1

In [20]:
Palindrome 'Am I a palindrome?'

0

Now, in `Palindrome` we decide to replace `Reverse` by `ReverBug`, a deliberately faulty and obtuse version.
Not only does it use an unnecessary loop, but it includes some (intentional) errors:

In [21]:
∇ rev ← ReverBug vec; from; to    ⍝ Intentionally faulty.
    rev ← (≢vec)⍴'?'
    from ← 0
    :Repeat
        from ← frm+1
        to ← (≢vec)-from
        rev[to] ← vec[from]
    :Until from=≢vec
∇

#### Let Us Create More Errors

First, let us execute the little function `Plus`, shown below, with incorrect arguments, so as to obtain a `LENGTH ERROR`:

In [22]:
]dinput
Plus ← {
    ⍺ + ⍵
}

In [23]:
5 1 7 Plus 6 2 9 4

LENGTH ERROR: Mismatched left and right argument shapes
Plus[1] ⍺+⍵
         ∧


As in the previous example, if you are following along with the examples in the Windows interpreter or in RIDE,
 - an error message is issued, and a trace window pops up;
 - `⎕SI:1` is now highlighted and displayed in the status bar, and `)si` contains one line; and
 - `⎕LC` contains the value `1` (the function is interrupted on line 1).

We shall intentionally ignore this error.
**Do not close** the trace window.

As a side comment, notice that if you defined the `Plus` dfn in a single line, for example with `Plus ← {⍺ + ⍵}`, then the trace window will *not* appear. In that case, you would have to redefine `Plus` to span multiple lines *or* rewrite it as a tradfn.

And now, let us execute `Palindrome` (after replacing the usage of `Reverse` with the faulty `ReverBug`).
An error is immediately reported:

In [24]:
]dinput
Palindrome ← {
    vector ← Upper ⍵
    torvec ← ReverBug vector
    torvec≡vector
}

In [25]:
Palindrome 'Was it a cat or a car I saw?'

VALUE ERROR: Undefined name: frm
ReverBug[4] from←frm+1
                 ∧


 - a second error message is issued;
 - in the bottom right corner of our session, we now have the information `⎕SI:3`;
 - `⎕LC` contains three values: `4 2 1`; and
 - we now have three stacked trace windows instead of one:
 
   - one window contains `Plus`, unchanged;
   - a second window contains `Palindrome`, in a different colour; and
   - on top of that, a third window contains `ReverBug`.
   
Although the three trace windows are stacked, you should only be able to view one at a time and, in this case, the `ReverBug` trace window should be the one that is visible right now.

First, let us consult the indicators we know.
`⎕DM` still shows the latest known error, and `⎕SI` gives the names of the three interrupted functions, the last one being displayed first:

```APL
      ⎕DM
┌───────────┬──────────────────────┬──────────────────┐
│VALUE ERROR│ReverBug[4] from←frm+1│                 ∧│
└───────────┴──────────────────────┴──────────────────┘
      ⎕SI
┌────────┬──────────┬────┐
│ReverBug│Palindrome│Plus│
└────────┴──────────┴────┘
      )si
#.ReverBug[4]*
#.Palindrome[2]
#.Plus[1]*
```

Notice that
 - the star next to `#.ReverBug[4]` means `ReverBug` is interrupted;
 - `#.Palindrome[2]` has no star to its right, it is "pending"; and
 - there is also a star next to `#.Plus[1]` because `Plus` was also interrupted.

As you can see, `)si` is a stack.
Starting from the bottom, it indicates that `Plus` was interrupted on line `[1]`.
Then `Palindrome[2]` has called `ReverBug`, and finally `ReverBug` caused an error in line `[4]`.

Both `⎕SI` and `⎕LC` report the most recently interrupted function first, followed by the others.

`Palindrome` has not itself generated an error: it is just waiting for the completion of `ReverBug`.
The `Palindrome` function is said to be *pending*.
For that reason
 - there is no star to the right of `Palindrome` in `)si`; and
 - the background of its trace window is a different colour from the others, if you are using the Windows interpreter.

#### Switch to Edit Mode

In this case, the error is obvious and we can try to correct it, but the text displayed in the trace window cannot be edited directly;
we must first turn the trace window into an edit window.

To do this, ensure that your cursor is over an empty space in the session window (it must not be over another name) and then:
 - just double-click (on nothing); or
 - press <kbd>Shift</kbd>+<kbd>Enter</kbd>.
 
Alternatively, you can switch focus to the trace window and place the cursor before the start of any function line and press <kbd>Shift</kbd>+<kbd>Enter</kbd>.

All three methods should make the colour of the trace window change, most likely from grey to white, if you are using the default colour scheme.

You can now modify the incorrect statement (change `frm` into `from`), and press <kbd>Esc</kbd> to fix the modification.
The system switches back to the original colour, showing the newly corrected statement.

#### Continue Execution

After making the correction, you can choose to either:
 - resume the function from the interruption point; or
 - continue by tracing its execution, statement by statement, to see what happens.

Let us choose the first option.
This can be achieved using one of the following methods:
 - in the session window, execute `→⎕LC`, which is equivalent to `→4 2 1` in this case.
The branch arrow ignores all but the first number, so the program resumes execution from line number 4 in the function at the top of the stack (`ReverBug` in this case); or
 - in the trace window toolbar, press the *continue execution of this thread* button. <!--figure-->The next two figures<!--Win_Continue_Execution_Button,RIDE_Continue_Execution_Button--> show what this button looks like for users of the Windows interpreter and of RIDE, respectively.

![*Continue execution of this thread* button in the Windows interpreter trace window.](res/Win_Continue_Execution_Button.png)

<br>

![*Continue execution of this thread* button in the RIDE trace window.](res/RIDE_Continue_Execution_Button.png)

The function resumes its execution but, unfortunately, a second error immediately appears:

```APL
INDEX ERROR
ReverBug[6] rev[to]←vec[from]
               ∧
```

The indicators are nearly the same; they now tell us that the function is no longer interrupted on line 4, but on line 6.

To diagnose the problem, we can look at the values of our local variables.
We can either double-click on their names (which will open each one in a new window), or just type their names in the session; let us use the latter approach:

```APL
      rev
ASITACARORACATISAW?
```

Something is wrong: the first letter (W) should be in the last position where the "?" is.

```APL
      ≢vec
19
```

This confirms we had an argument with 19 letters.

```APL
      from
19
```

And we are trying to move the 19th letter...

```APL
      to
0
```

to the last index, but it being equal to zero causes an error.

Our statement in line [5] calculated the wrong index!
It would be insufficient to correct the statement and continue with the current execution, because the variables are already wrong.
A better solution would be to correct line [5], exit from `ReverBug` and restart where `Palindrome` called it.

Let us modify the 5th statement of `ReverBug` to be `to ← (1+≢vec)-from`.
Now, if we want to return to the calling function, we cannot execute `)reset` nor `→`, because that would cause a complete exit both from `ReverBug` and from `Palindrome`.
The only way back is to ensure that the `ReverBug` trace window has the focus and then press <kbd>Esc</kbd> to get out of `ReverBug` and return to `Palindrome`.

We are now back in the callling environment:

```APL
      )si
#.Palindrome[2]*
#.Plus[1]*
```

Notice that `ReverBug` is no longer present in the state indicator and that the asterisk is now displayed alongside `Palindrome`, because `Palindrome` is now the interrupted function.
Then, resume the execution of `Palindrome` exactly as we restarted `ReverBug`:
 - execute `→⎕LC` in the session; or
 - press the *continue execution of this thread* button in the trace window toolbar.

```APL
      →⎕LC
1
```

If we run `→⎕LC` (or press the *continue execution* button, for that matter) the result we get back is 1, because the sentence `'Was it a car or a cat I saw?'` is a palindrome.
Despite `Palindrome` having returned its result, we still have an old interrupted function:

```APL
      )si
#.Plus[1]*

      ⎕DM
┌───────────┬─────────────────────────────┬────────────────┐
│INDEX ERROR│ReverBug[6] rev[to]←vec[from]│               ∧│
└───────────┴─────────────────────────────┴────────────────┘
```

As you can see, `⎕DM` does not report the stack of errors, just the last one that occurred.

The best thing we can do now to restore a clean environment is to execute:

```APL
      )reset
```

### Information and Actions

#### Indicators

When an error occurs, a number of program status indicators are available fro the developer's use.
Not all of them will be described here; we shall just explore the most useful ones.

| Code | Name | Description |
| :- | :- | :- |
| `)si` | *State indicator* | List of suspended functions with line numbers and state. |
| `⎕SI` | *State indicator* | Nested vector of suspended or pending functions names. |
| `⎕DM` | *Diagnostic message* | 3-item nested vector which reports the message associated with the latest error. |
| `⎕LC` | *Line counter* | Numeric vector containing the line numbers of functions that are pending or suspended and waiting for execution, the last one being displayed first – the same order as `⎕SI`. |
| `⎕EN` | *Event number* | Every type of error or exception is identified by a number. The number of the latest error is reported by `⎕EN`. |

Here are some very common *event numbers*; they will be explained in the next section:

| `⎕EN` | Message |
| :- | :- |
| 1 | `WS FULL` |
| 2 | `SYNTAX ERROR` |
| 3 | `INDEX ERROR` |
| 4 | `RANK ERROR` |
| 5 | `LENGTH ERROR` |
| 6 | `VALUE ERROR` |
|11 | `DOMAIN ERROR` |

A full list of *event numbers* is given in [an appendix](./Appendices.ipynb#Event-Numbers).

#### Some Possible Actions

 - `→⎕LC` continues the interrupted function from the very statement on which it was interrupted (same as pressing the *continue execution of this thread* button);
 - `→0` quits the interrupted function and returns to its calling environment, which may be the session, or a calling function (same as putting focus on the trace window of the interrupted function and pressing <kbd>Esc</kbd>). In the latter case, execution will be resumed in the calling function;
 - `→` quits the interrupted function and all its calling functions, and returns to the session. If other functions are suspended, they remain in the *state indicator*; and
 - `)reset` quits **all** suspended functions, resetting the *state indicator*. `⎕SI`, `⎕DM`, and `⎕LC` are reset to empty vectors and `⎕EN` is set to zero.

### Why Should You Reset Your State Indicator?

Let us pretend for a second that you were working in the session, trying to figure out how to solve the exercises from [the section on dfns](./User-Defined-Functions.ipynb#Exercises-on-Dfns).
In particular, you were trying to solve the exercise where you have to implement a function that produces a series of integers starting on one argument and going up to the other one.

First things first, you create the variables that tell you where the sequence should start and end:

In [26]:
from ← 17
to ← 29

Then you start trying to figure out how many numbers you should generate, and how to actually generate them:

In [27]:
to-from

12

In [28]:
⍳to-from

1 2 3 4 5 6 7 8 9 10 11 12

In [29]:
from+⍳to-from

18 19 20 21 22 23 24 25 26 27 28 29

In [30]:
¯1+from+⍳to-from

17 18 19 20 21 22 23 24 25 26 27 28

Eventually you reach this expression and feel good about it, so you decide to save it as a short dfn:

In [31]:
To ← {¯1+⍺+⍳⍵-⍺}

In [32]:
17 To 29

17 18 19 20 21 22 23 24 25 26 27 28

The exercise is solved and you proceed to programming something else, so you decide to implement a `Reverse` function, but alas you make a flawed implementation, like the one in `ReverBug`:

In [33]:
∇ rev ← ReverBug vec; from; to    ⍝ Still intentionally faulty.
    rev ← (≢vec)⍴'?'
    from ← 0
    :Repeat
        from ← from+1
        to ← (≢vec)-from
        rev[to] ← vec[from]
    :Until from=≢vec
∇

When you call it, you realise you made a mistake, because the function throws an error:

In [34]:
ReverBug 17 To 29

INDEX ERROR
ReverBug[6] rev[to]←vec[from]
               ∧


You look around for a bit and find your mistake, as the fifth statement should have read `to ← (1+≢vec)-from` instead of the current `to ← (≢vec)-from`.
You correct your mistake and fix the function, but then don't reset the stack because you see no point in it.
Just a little while latter, you reuse your `To` function and notice that it looks weird:

In [35]:
17 To 29

17 18 19 20 21 22 23 24 25 26 27 28

The sequence of numbers produced should go up to, and including, 29, and it currently stops at 28.
So, you decide to go back and play a bit more with the expression you had before defining the `To` function:

```APL
      ¯1+from+⍳to-from
DOMAIN ERROR
      ¯1+from+⍳to-from
              ∧
      to from
0 12
```

The variables `to` and `from` you were playing with are not visible in your session because they are being shadowed ("hidden" or "covered") by the local variables inside `ReverBug`.

As soon as we reset the *state indicator*, the local variables of `ReverBug` disappear, and your little expression can be used again:

```APL
      )reset
      ¯1+from+⍳to-from
17 18 19 20 21 22 23 24 25 26 27 28
```

You then proceed to understanding that you are generating one too few numbers, and manage to come up with the correct expression for your `To` function:

In [36]:
¯1+from+⍳1+to-from

17 18 19 20 21 22 23 24 25 26 27 28 29

In [37]:
To ← {¯1+⍺+⍳1+⍵-⍺}

So, having a "dirty" state indicator can easily lead to a lot of confusion.

You should also be aware that because APL preserves the full context in which a function stopped or crashed, all the local variables are retained too; this may require a lot of memory space!

<!--begin tip-->
***Recommendation***:

 > When a function crashes, always try to clear the stack if you can:
 >   - try to correct the problem and resume function execution; or
 >   - if that is impossible, abort it and reset the execution context by executing `)reset`.
 >
 > If you can't fix it *right now*, Dyalog will allow you to `)save` the workspace and come back to it later.
<!--end-->

## Most Frequent Error Messages

### Execution Errors

For each common error, we shall give you the most probable reason for the error, some rare cases which may be difficult to debug, and suggest some immediate experiments that you can carry out to help diagnose the problem.

Among the six most common errors, four are really easy to diagnose, but the last two are a bit more complex.

#### `VALUE ERROR` (`⎕EN = 6`)

You used a name which does not have a value; it does not represent a variable, a function, or an operator.

Here are some possible reasons:
 - perhaps you misspelled the variable (or function) name, entered the incorrect mixture of upper and lower case characters, or confused the letter "O" with the digit "0", etc;
 - you tried to use the result of a function which actually produces no result (see [the section the return value of functions](./User-Defined-Functions.ipynb#Why-Should-a-Function-Return-a-Result);
 - you tried to reference one of the local variables of a function before it was assigned a value – remember that all variables referenced in the header of a tradfn are local, and so are all the variables defined inside dfns; and
 - an erroneous tradfn header, as explained below.

Imagine you intended to create a function named `Travel`, with two arguments named `west` and `east`.
When you entered the header, you forgot the space between `Travel` and `east`, so that the function now looks like this:

In [38]:
∇ z ← west Traveleast
    z ← west-east
∇

Instead, you just defined a monadic function named `west`, with a right argument named `Traveleast`.
When you try to use `Travel`, it does not exist, and you get an error:

In [39]:
0 Travel 1

VALUE ERROR: Undefined name: Travel
      0 Travel 1
        ∧


If you suspect that this has occurred, and you can remember the syntax of the function, check to see if you have a different function whose name is the same as your intended left argument.
For example, below we can find a function named "`west`" instead of "`Travel`":

In [41]:
)fns

DISPLAY Interlace Palindrome Plus ReverBug Reverse To Upper west

If you do not spot it immediately, and if you are using the Windows interpreter, you can use the *search* tool (generally available in the toolbar as the magnifying lens or under "Tools" &#8680; "Search...").
Once the window opens, you can type the name that you want to search for and then click "Find Now".
You should then discover what the name of the misspelled function really is, as shown in <!--figure-->the figure below<!--Win_Search_Travel-->.

![The "Search" window of the Windows interpreter.](res/WIN_Search_Travel.png)

#### `LENGTH ERROR` (`⎕EN = 5`)

A `LENGTH ERROR` usually involves two arguments with inconsistent shapes for a given operation.
For example:

In [42]:
4 6 8 + 3 7

LENGTH ERROR: Mismatched left and right argument shapes
      4 6 8+3 7
           ∧


<!--begin tip-->
***Suggestion***:

 > If an operation like `X+Y` causes a `LENGTH ERROR`, look at the shapes of both arguments `X` and `Y`, and see if they are compatible with one another.
<!--end-->

Also remember that:
 - a matrix with only one row looks like (but is not) a vector and a matrix/vector with only one element looks like (but is not) a scalar: see [this subsection](./Data-and-Variables.ipynb#Beware); and
 - an incorrectly specified axis may also cause such an error.

To exemplify a `LENGTH ERROR` with an incorrect axis specification, let us define

In [43]:
⎕← MatA ← 3 4⍴⍳12

1 2 3 4
5 6 7 8
9 10 11 12

In [46]:
⎕← MatB ← 5 4⍴⍳20

1 2 3 4
 5 6 7 8
 9 10 11 12
13 14 15 16
17 18 19 20

Both `MatA,MatB` and `MatA,[2]MatB` cause a `LENGTH ERROR` but both `MatA⍪MatB` and `MatA,[1]MatB` will work:

In [48]:
MatA,MatB

LENGTH ERROR
      MatA,MatB
          ∧


In [49]:
MatA,[2]MatB

LENGTH ERROR
      MatA,[2]MatB
          ∧


In [50]:
MatA⍪MatB

1 2 3 4
 5 6 7 8
 9 10 11 12
 1 2 3 4
 5 6 7 8
 9 10 11 12
13 14 15 16
17 18 19 20

In [51]:
MatA,[1]MatB

1 2 3 4
 5 6 7 8
 9 10 11 12
 1 2 3 4
 5 6 7 8
 9 10 11 12
13 14 15 16
17 18 19 20

#### `RANK ERROR` (`⎕EN=4`)

A `RANK ERROR` means that you tried to do something which is not consistent with the *rank* of an array.
This may involve one or more variables.
Let us recall our `forecast` matrix and use it to show some common examples of this type of error:

In [52]:
⎕RL ← 73
⎕←forecast ← 10×?4 6⍴55

90 160 420 500 20 30
110 450 170 370 290 360
340 190 320 120 510 370
150 460 240 520 490 280

In [59]:
forecast[3 5]

INDEX ERROR
      forecast[3 5]
              ∧


The indexing above is incorrect because `forecast` is a matrix, not a vector.
A semicolon is needed somewhere: `[;3 5]`, `[3;5]`, or `[3 5;]`.

In [60]:
forecast + ⍳6

RANK ERROR: Mismatched left and right argument ranks
      forecast+⍳6
              ∧


The above cannot work, even if `forecast` had only one row and looked very much like a vector.

A `RANK ERROR` is also issued when an *axis* is not correct.
For example:

In [57]:
MatA,[3]MatB

RANK ERROR: Invalid axis
      MatA,[3]MatB
          ∧


This cannot work because `MatA` does not have 3 dimensions.
Similarly,

In [58]:
1 2 3 4,[2]5

RANK ERROR: Invalid axis
      1 2 3 4,[2]5
             ∧


cannot work because a vector only has one dimension.

Sometimes incorrect syntax can also lead to a `RANK ERROR`.
For example, to concatenate the two matrices shown above, the correct syntax is `MatA,[1]MatB`, but you could have typed that as

In [55]:
MatA[1],MatB

RANK ERROR
      MatA[1],MatB
          ∧


Notice the misplaced comma, which leads APL to try and calculate `MatA[1]`, which fails because `MatA` is a matrix and not a vector.

<!--begin tip-->
***Suggestion***:

 > If a statement causes a `RANK ERROR`, look at the shapes of all the arrays involved in the expression.
<!--end-->

#### `INDEX ERROR` (`⎕EN=3`)

You tried to index a variable, but your index was wrong.
For example, you tried to get the 20th element of a vector which only has 17 elements.

This occurs when the index (or one of the items of the index) is:
 - smaller than or equal to zero;
 - greater than the length of the variable, along the specified axis; or
 - not an integer.

In [56]:
forecast[5;5]

INDEX ERROR
      forecast[5;5]
              ∧


This doesn't work because `forecast` has 6 columns but only 4 rows: the second index is good but the first one is not.

<!--begin tip-->
***Suggestion***:

 > If the error occurs in an indexing operation like `var[indexA;indexB;...]`, look at the shape of `var` and check if `indexA`, `indexB`, etc., contain values which are compatible with the size of `var`.
<!--end-->